In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [ ]:
#Load Pre-Processed Train / Test Data

train_df = pd.read_csv('Train-Data.csv')
test_df = pd.read_csv('Test-Data.csv')

print(f'Train Length: {len(train_df)}, Test Length: {len(test_df)}')

In [ ]:
#Prepare Data For ML input
trainx = np.array(train_df.iloc[:,:-1].values.tolist())
trainy = np.array(train_df['label'].tolist())

testx = np.array(test_df.iloc[:,:-1].values.tolist())
testy = np.array(test_df['label'].tolist())

In [ ]:
#Validate Correct Input Shapes
print(f'Trainx: {trainx.shape}, Trainy: {trainy.shape}')
print(f'Testx: {testx.shape}, Testy: {testy.shape}')

In [ ]:
#Encode Classification Labels
le = LabelEncoder()

le.fit(trainy)

trainy = le.transform(trainy)
testy = le.transform(testy)

In [ ]:
#Build Fully Connected Neural Network Model
FCNN = Sequential()
FCNN.add(Dense(32, input_dim=7, activation='relu'))
FCNN.add(Dense(64, activation='relu'))
FCNN.add(Dense(22, activation='softmax'))
FCNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
FCNN.summary()

In [ ]:
FCNN.fit(trainx, trainy, epochs=50, batch_size=32)

In [ ]:
FCNN_Preds = FCNN.predict(testx)
FCNN_Preds = np.array([np.argmax(i) for i in FCNN_Preds])

In [ ]:
def get_performance_metrics(preds):
    metrics = {}
    metrics['acc'] = accuracy_score(testy, preds)
    metrics['prec'] = precision_score(testy, preds, average='macro')
    metrics['rec'] = recall_score(testy, preds, average='macro')
    metrics['f1'] = f1_score(testy, preds, average='macro')
    metrics['cm'] = confusion_matrix(testy, preds)
    return metrics

In [ ]:
FCNN_Metrics = get_performance_metrics(FCNN_Preds)
print(FCNN_Metrics['acc'])

In [ ]:
#Build CNN
CNN = Sequential()
CNN.add(Conv1D(filters=32, kernel_size=3, input_shape=(7, 1), activation='relu'))
CNN.add(Flatten())
CNN.add(Dense(128, activation='relu'))
CNN.add(Dense(22, activation='softmax'))
CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
trainx_reshaped = trainx.reshape(trainx.shape[0], 7, 1)
testx_reshaped = testx.reshape(testx.shape[0], 7, 1)
CNN.fit(trainx_reshaped, trainy, epochs=50, batch_size=32)

In [ ]:
CNN_Preds = CNN.predict(testx)
CNN_Preds = np.array([np.argmax(i) for i in CNN_Preds])

In [ ]:
CNN_Metrics = get_performance_metrics(CNN_Preds)